# Machine Learning - Practical 4 - Deep Learning VS Trees


Names: {YOUR NAMES}  
Summer Term 2023   
Due Date: Tuesday, June 13, 2pm

In this practical we are going to use a tabular dataset. We will test two different approaches - forests and neural networks and compare performance. We are also going to learn how to make trees interpretable.

To prepare this tutorial we used [this paper](https://arxiv.org/pdf/2207.08815.pdf) with its [repository](https://github.com/LeoGrin/tabular-benchmark).

For explained variance in trees, you can read more [here](https://scikit-learn.org/0.15/auto_examples/ensemble/plot_gradient_boosting_regression.html#example-ensemble-plot-gradient-boosting-regression-py).


In [1]:
%matplotlib inline


import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import pickle
import os

from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import recall_score, precision_score, accuracy_score
from sklearn.metrics import classification_report

from PIL import Image
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset, Subset
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torchvision.utils import make_grid
torch.manual_seed(42) # Set manual seed

In [2]:
# DO NOT CHANGE
use_cuda = True
use_cuda = False if not use_cuda else torch.cuda.is_available()
device = torch.device('cuda:0' if use_cuda else 'cpu')
torch.cuda.get_device_name(device) if use_cuda else 'cpu'
print('Using device', device)

Using device cuda:0


## Load, clean and split the tabular dataset

We use the preprocessing pipeline from [Grinsztajn, 2022](https://arxiv.org/pdf/2207.08815.pdf).

**No missing data**    

Remove all rows containing at least one missing entry.    

*In practice people often do not remove rows with missing values but try to fill missing values in a column with the mean or median values for numerical data and mode or median values for categorical data. Sometimes even simple prediction models are used to fill in the gaps but we will remove rows or columns with missing values for the sake of simplicity*

**Balanced classes**   

For classification, the target is binarised if there are multiple classes, by taking the two most numerous classes, and we keep half of samples in each class.

**Low cardinality categorical features**   

Remove categorical features with more than 20 items. 

**High cardinality numerical features**   

Remove numerical features with less than 10 unique values. Convert numerical features with 2 unique values to categorical.

**Data description:**  
Data reported to the police about the circumstances of personal injury road accidents in Great Britain from 1979. This version includes data up to 2015. We will try to predict the sex of the driver based on the data provided.

In [3]:
## In case you have any issues with loading the pickle file
## check that your pandas version is 1.4.1
## or just simply run:
## pip install pandas==1.4.1

with open('/kaggle/input/adopted-road-safety/adopted_road_safety.pkl', 'rb') as f:
    dataset = pickle.load(f)

In [4]:
dataset

,Accident_Index,Vehicle_Reference_df_res,Vehicle_Type,Towing_and_Articulation,Vehicle_Manoeuvre,Vehicle_Location-Restricted_Lane,Junction_Location,Skidding_and_Overturning,Hit_Object_in_Carriageway,Vehicle_Leaving_Carriageway,...,Age_Band_of_Casualty,Casualty_Severity,Pedestrian_Location,Pedestrian_Movement,Car_Passenger,Bus_or_Coach_Passenger,Pedestrian_Road_Maintenance_Worker,Casualty_Type,Casualty_Home_Area_Type,Casualty_IMD_Decile
0,201501BS70001,1,19.0,0.0,9.0,0.0,8.0,0.0,0.0,0.0,...,7.0,3,5.0,1.0,0.0,0.0,2.0,0,NaN,NaN
1,201501BS70002,1,9.0,0.0,9.0,0.0,8.0,0.0,0.0,0.0,...,5.0,3,9.0,9.0,0.0,0.0,2.0,0,1.0,3.0
2,201501BS70004,1,9.0,0.0,9.0,0.0,2.0,0.0,0.0,0.0,...,6.0,3,1.0,3.0,0.0,0.0,2.0,0,1.0,6.0
3,201501BS70005,1,9.0,0.0,9.0,0.0,2.0,0.0,0.0,0.0,...,2.0,3,5.0,1.0,0.0,0.0,2.0,0,1.0,2.0
4,201501BS70008,1,1.0,0.0,18.0,0.0,8.0,0.0,0.0,0.0,...,8.0,2,0.0,0.0,0.0,0.0,0.0,1,1.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
363238,2015984141415,13,9.0,0.0,18.0,0.0,0.0,0.0,0.0,5.0,...,1.0,3,0.0,0.0,2.0,0.0,0.0,9,1.0,NaN
363239,2015984141415,13,9.0,0.0,18.0,0.0,0.0,0.0,0.0,5.0,...,5.0,3,0.0,0.0,0.0,0.0,0.0,9,1.0,2.0
363240,2015984141415,13,9.0,0.0,18.0,0.0,0.0,0.0,0.0,5.0,...,4.0,3,0.0,0.0,0.0,0.0,0.0,9,2.0,5.0
363241,2015984141415,13,9.0,0.0,18.0,0.0,0.0,0.0,0.0,5.0,...,6.0,3,0.0,0.0,0.0,0.0,0.0,9,3.0,NaN


In [5]:
target_column = 'Sex_of_Driver'
test_size = 0.2
random_state = 42

In [6]:
def remove_nans(df):
    '''
    this function removes rows with nans 
    '''
    df = df.dropna()
        
    # TODO
    return df


def numerical_to_categorical(df, n=2, ignore=[target_column]):
    '''
    change the type of the column to categorical 
    if it has <= n unique values
    '''
    
    for col in df.columns:
        if col not in ignore:
            count = df[col].unique().size
            if count<=n:
                df[col] = df[col].astype("category")
            
    return df


def remove_columns_by_n(df, n=10, condition=np.number, direction='less', 
                        ignore=[target_column]):
    '''
   
    Remove columns with more or less than n unique values. 
    Usually it makes sense to apply this function to columns with categorical values (see below where it is called).
    With the default values we remove all numerical columns which have less than 10 unique values (except for the target_column).
    '''
    # TODO
    if direction=='less':
        for col in df.columns:
            if col not in ignore:
                count = df[col].unique().size
                if count < n :
                    df = df.drop(col, axis = 1)
    else:
        for col in df.columns:
            if col not in ignore:
                count = df[col].unique().size
                if count > n :
                    df = df.drop(col, axis = 1)
        
        
            
    return df

In [7]:
df = dataset
df = remove_nans(df)
df = numerical_to_categorical(df, n=2, ignore=[target_column])
df = remove_columns_by_n(df, n=10, condition=np.number, direction='less', 
                         ignore=[target_column])
df = remove_columns_by_n(df, n=40, condition='category', direction='more', 
                         ignore=[target_column])
assert not df.isna().any().any(), 'There are still nans in the dataframe'

df

/tmp/ipykernel_28/3679314217.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = df[col].astype("category")
/tmp/ipykernel_28/3679314217.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = df[col].astype("category")
/tmp/ipykernel_28/3679314217.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_g

,Vehicle_Type,Vehicle_Manoeuvre,Vehicle_Location-Restricted_Lane,Hit_Object_in_Carriageway,Hit_Object_off_Carriageway,Sex_of_Driver,Age_Band_of_Driver,Police_Force,Number_of_Casualties,Casualty_Reference,Age_Band_of_Casualty,Pedestrian_Location,Pedestrian_Movement,Casualty_Type,Casualty_IMD_Decile
2,9.0,9.0,0.0,0.0,0.0,1.0,6.0,1.0,1.0,1,6.0,1.0,3.0,0,6.0
6,3.0,18.0,0.0,0.0,0.0,1.0,7.0,1.0,1.0,1,7.0,0.0,0.0,3,5.0
7,19.0,6.0,0.0,0.0,0.0,1.0,7.0,1.0,1.0,1,7.0,0.0,0.0,3,5.0
8,9.0,9.0,0.0,0.0,0.0,1.0,7.0,1.0,1.0,1,7.0,0.0,0.0,1,8.0
12,9.0,13.0,0.0,0.0,0.0,2.0,7.0,1.0,1.0,1,9.0,0.0,0.0,5,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
328127,9.0,5.0,0.0,0.0,0.0,2.0,10.0,55.0,2.0,2,7.0,0.0,0.0,9,6.0
328128,9.0,18.0,0.0,0.0,0.0,1.0,4.0,55.0,2.0,1,4.0,0.0,0.0,9,10.0
328129,9.0,18.0,0.0,0.0,0.0,1.0,4.0,55.0,2.0,2,7.0,0.0,0.0,9,6.0
328130,9.0,3.0,0.0,0.0,0.0,2.0,7.0,55.0,2.0,1,4.0,0.0,0.0,9,10.0


In [8]:
# TODO : make train-test split from the dataframe using the parameters above
# expected results variable names - train_X, test_X, train_y, test_y

X = df.drop(target_column, axis = 1)
Y = df[target_column]

train_X, test_X, train_y, test_y  = train_test_split(X, Y, test_size=test_size, random_state=random_state)

**TODO :**  

* Did you split the dataset in a stratified manner or not? Why?
* How did the dataset dimensions change after preprocessing?
* How many unique values are in the response variable? 

## Task 1: Create a GradientBoostingClassifier

In [9]:
## TODO : define the GradientBoostingClassifier, 
## train it on the train set and predict on the test set
GBC = GradientBoostingClassifier()



In [ ]:
## TODO : print  accuracy, precision, recall
## Hint : use functions from sklearn metrics
GBC.fit(train_X, train_y)
predictions = GBC.predict(test_X)
accuracy = accuracy_score(test_y, predictions)
precision = precision_score(test_y, predictions, average='weighted')
recall = recall_score(test_y, predictions, average='weighted')
print('Accuracy:', accuracy)
print('Precision:', precision)
print('Recall:', recall)



In [11]:
## TODO : Write a function which iterates over trees_amount, 
## train a classifier with a specified amount of trees and print accuracy, precision, and recall.
## Note: the calculations may take several minutes (depending on the computer efficiency).

def trees_amount_exploration(train_X, train_y, test_X, test_y, trees_amount=[1, 20, 50, 100]):
    #TODO
    for trees in trees_amount:
        model = GradientBoostingClassifier(n_estimators=trees)
        model.fit(train_X, train_y)
        predictions = model.predict(test_X)
        accuracy = accuracy_score(test_y, predictions)
        precision = precision_score(test_y, predictions, average='weighted')
        recall = recall_score(test_y, predictions, average='weighted')
        
        print('trees_amount:', trees, '\n')
        
        print('Accuracy:', accuracy)
        print('Precision:', precision)
        print('Recall:', recall, '\n \n')
        


In [ ]:
trees_amount_exploration(train_X, train_y, test_X, test_y)

In [12]:
## TODO : Write a function which iterates over the learning rate, 
## train a classifier with a specified amount of trees and print accuracy, precision, and recall.
## Note: the calculations may take several minutes (depending on the computer efficiency).

def learning_rate_exploration(train_X, train_y, test_X, test_y, learning_rates = [0.1, 0.2, 0.3, 0.4, 0.5], trees_amount=100):
    #TODO
    for learning_rate in learning_rates:
        model = GradientBoostingClassifier(learning_rate=learning_rate)
        model.fit(train_X, train_y)
        predictions = model.predict(test_X)
        
        accuracy = accuracy_score(test_y, predictions)
        precision = precision_score(test_y, predictions, average='weighted')
        recall = recall_score(test_y, predictions, average='weighted')
        
        print('Learning rate:', learning_rate, '\n')
        
        print('Accuracy:', accuracy)
        print('Precision:', precision)
        print('Recall:', recall, '\n \n')

In [ ]:
learning_rate_exploration(train_X, train_y, test_X, test_y)

In [13]:
## TODO : Write a function which iterates over different depths, 
## train a classifier with a specified depth and print accuracy, precision, and recall
## Set trees_amount= 50 to make the calculations faster
## Note: the calculations may take several minutes (depending on the computer efficiency).

def max_depth_exploration(train_X, train_y, test_X, test_y, depths=[1,2,3,5]):
    # TODO
     for depth in depths:
        model = GradientBoostingClassifier(max_depth=depth)
        model.fit(train_X, train_y)
        
        predictions = model.predict(test_X)
        accuracy = accuracy_score(test_y, predictions)
        precision = precision_score(test_y, predictions, average='weighted')
        recall = recall_score(test_y, predictions, average='weighted')
        
        print('Depth:', depth, '\n')
        
        print('Accuracy:', accuracy)
        print('Precision:', precision)
        print('Recall:', recall, '\n \n')

In [ ]:
max_depth_exploration(train_X, train_y, test_X, test_y)

**TODO :**   

* How does the max_depth parameter influence the results? 
* How does the learning rate influence the results?
* How does the number of trees in the ensemble influence the results?
* Try to improve the accuracy by combining different max_depth, learning rate and number of trees. How well does your best model perform?

In [ ]:
## TODO -  sklearn trees have the attribute feature_importances_
## make a plot, to show relative importance (maximum is 1) of your classifier and
## order features from most relevant feature to the least relevant in the plot

def plot_explained_variance(clf, X):
    # TODO

In [ ]:
## TODO : display the plot


**TODO :** Interpret the plot.

**TODO (optional):** Try to remove the least-important features and see what happens. Does to quality improve or degrade? Why? 

## Prepare for deep learning
### Add all the necessary training functions 
*You can reuse them from previous practical exercises*

In [9]:
## TODO write a function that calculates the accuracy
## Hint - you can use yours from practical 3 

def accuracy(correct, total): 
    """
    function to calculate the accuracy given the
        correct: number of correctly classified samples
        total: total number of samples
    returns the ratio
    """
    ratio = 100 * (correct.item())/total 
    return ratio
    

In [20]:
## TODO : Define a train and validation functions here
## Hint - you can use yours from practical 3 

def train(dataloader, optimizer, model, loss_function):
    """ method to train the model """

    # TODO: refine the training function from above
  # it should contain:
  # - saving of losses
  # - calculation of accuracy
  # - returning the mean loss and accuracy
    losses = []
    model.train()
    sum_loss = 0.0
    total = 0
    correct = 0
    
    
    for i, (x,y) in enumerate(dataloader):
        x= x.to(device)
        y = y.to(device).squeeze()
                        
        optimizer.zero_grad()
        
        y_pred = model(x)
        _, predicted = torch.max(y_pred.data,1)
        

        # Compute loss
        loss = loss_function(y_pred, y)
        

        # Backward pass -> calculate gradients, update weights
        loss.backward()
        optimizer.step()
        
        losses.append(loss.item())
        total += y.size(0)
        sum_loss += loss.item()
        correct += (predicted == y).sum()
    
    n = len(losses)
    mean_loss= sum_loss/n
    acc = accuracy(correct, total)
    
    return mean_loss, acc


def validate(dataloader, model, loss_fn, device):
    """ method to compute the metrics on the validation set """
    # Todo
    model.eval()  # Set the model in evaluation mode
    val_loss = 0.0
    correct = 0
    total=0
    
    with torch.no_grad():
        for i, (x_val, y_val) in enumerate(dataloader):
            x_val = x_val.to(device)
            y_val = y_val.to(device)

            y_pred_val = model(x_val)
            _, predicted = torch.max(y_pred_val.data,1)
            loss_val = loss_function(y_pred_val.squeeze(), y_val)     

            val_loss += loss_val.item()
            total += y_val.size(0)
            correct += ( predicted == y_val).sum()

        val_loss /= len(dataloader)
        acc = accuracy(correct, total)
    
    return val_loss, acc
    

In [11]:
#TODO write a run_training function that 
# - calls the train and validate functions for each epoch
# - saves the train_losses, val_losses, train_accs, val_accs as arrays for each epoch
## Hint - you can use yours from practical 3 
from tqdm import tqdm


def run_training(model, optimizer, loss_function, device, num_epochs, train_dataloader, val_dataloader):
    """ method to run the training procedure """
    # TODO
    train_losses=[]
    val_losses = []
    train_accs = []
    val_accs = []
    for epoch in tqdm(range(int(num_epochs)), desc = 'Training Epochs'):
        train_loss=train(train_dataloader, optimizer, model, loss_function)[0]
        train_acc= train(train_dataloader, optimizer, model, loss_function)[1]
        val_loss = validate(test_dataloader,model,loss_function,device)[0]
        val_acc= validate(test_dataloader,model,loss_function,device)[1]
            
        train_losses.append(train_loss)
        val_losses.append(val_loss)
        train_accs.append(train_acc)
        val_accs.append(val_acc)
        
        print('Epoch: {}. Loss: {}. Accuracy: {}'.format(epoch, train_loss, train_acc))
        
    return train_losses, val_losses, train_accs, val_accs
        

In [ ]:
# TODO write a plot function 
## Hint - you can use yours from practical 2 or 3 #



### Convert a pandas dataframe to a PyTorch dataset

In [12]:
## TODO : Define the dataset, apply normalization in the getitem method
## Hint : you can use/adapt your code from practical 2
class TabularDataset(torch.utils.data.Dataset):
    def __init__(self, df_x, df_y, mean=None, std=None, normalise=True):
        '''
        TODO: save params to self attributes, 
        x is data without target column
        y is target column
        transform df to_numpy
        ''' 
        self.x = torch.tensor(df_x.values, dtype=torch.float32)
        self.y = torch.tensor(df_y.values, dtype=torch.float32)
        self.mean = mean
        self.std = std
        self.normalise = normalise
    
    def __len__(self):
        # TODO: return the length of the whole dataset
        return len(self.y)

    
    def __getitem__(self, index):
        ## TODO: return X, y by index, normalized if needed
        X = self.x[index]
        y = self.y[index]
        if self.normalise and self.mean is not None and self.std is not None:
            X = (X- self.mean) / self.std
        return X,y

In [13]:
## TODO : calculate mean and std for the train set
## Hint : be careful with categorical values. Convert them them to numerical 
## Hint : the response variable should be of datatype integer

train_y = pd.get_dummies(train_y)
test_y = pd.get_dummies(test_y)

cat_columns = train_X.select_dtypes(['category']).columns
train_X[cat_columns] = train_X[cat_columns].apply(lambda x: x.cat.codes)

tmp_dataset = TabularDataset(train_X, train_y, normalise= False)

mean = torch.mean(tmp_dataset.x, axis=0)
std = torch.std(tmp_dataset.x, axis=0)

print(mean)
print(std)
        

tensor([ 9.4492, 11.9831,  0.0555,  0.2947,  0.3868,  6.9186, 20.2898,  1.9120,
         1.4448,  6.4592,  0.2666,  0.1767,  7.8492,  4.8346])
tensor([ 5.8374,  5.9860,  0.6357,  1.5907,  1.8041,  1.7337, 17.1985,  1.3433,
         0.8988,  2.1332,  1.2212,  0.9777,  6.2523,  2.8173])


In [14]:
# TODO : define new datasets with mean, std and normalise=True
# be careful with the labels, they should start from 0!
conductor_train = TabularDataset(train_X, train_y, mean=mean, std=std, normalise=True)
conductor_test = TabularDataset(test_X, test_y,mean=mean,std=std, normalise=True)

## TODO : define dataloaders, with specified batch size and shuffled

batch_size = 256
train_dataloader = DataLoader(conductor_train,batch_size=batch_size, shuffle=True)
test_dataloader =DataLoader(conductor_test,batch_size=batch_size, shuffle=True)


## Logistic regression

In [21]:
class LR(torch.nn.Module):
    """
    The logistic regression model inherits from torch.nn.Module 
    which is the base class for all neural network modules.
    """
    def __init__(self, input_dim, output_dim):
        """ Initializes internal Module state. """
        super(LR, self).__init__()
        # TODO define linear layer for the model
        self.linear = nn.Linear(input_dim, output_dim)
        

    def forward(self, x):
        """ Defines the computation performed at every call. """
        # What are the dimensions of your input layer?
        x = x.to(torch.float32)
        #x = x.view(x.size(0), -1)
        # TODO run the data through the layer
        outputs = torch.sigmoid(self.linear(x))
        
        return outputs

In [22]:
## TODO define model, loss and optimisers
## don't forget to move everything for the correct devices
## 
lr=0.001
output_dim = 1
input_dim = train_X.shape[1]
linear_regression = LR(input_dim, output_dim)
linear_regression.to(device)
linear_regression.train()

optimizer = torch.optim.Adam(linear_regression.parameters(), lr=lr)
loss_function = nn.CrossEntropyLoss()



In [24]:
## TODO train the network
num_epochs = 30
run_training(linear_regression, optimizer,loss_function, device, num_epochs, train_dataloader, test_dataloader)

Training Epochs:   0%|          | 0/30 [00:00<?, ?it/s]


RuntimeError: 0D or 1D target tensor expected, multi-target not supported

In [ ]:
## todo - plot losses and accuracies
plot('Epoch vs. Loss', 'Loss', train_losses_lr, val_losses_lr)

In [ ]:
plot('Epoch vs. Accuracy', 'Accuracy', train_accs_lr, val_accs_lr)


## Create a simple MLP

As the default tree has 3 layers, let's make a MLP with 3 linear layers and ReLU.
Please notice that making convolutions on tabular data does not make much sense even though it is technically possible.   

**TODO :** Explain why making convolutions on tabular data does not make much sense. Why do we use an MLP, not a CNN from the previous homework?

In [ ]:
class TabularNetwork(torch.nn.Module):
    def __init__(self, input_dim, output_dim):
        """ Initializes internal Module state. """
        super(TabularNetwork, self).__init__()
        self.network = nn.Sequential(
            # TODO : define 3 linear layer with sizes 
            # input_dim -> input_dim // 2 -> output_dim
            # using ReLU as nonlinearity
            
        )
      

    def forward(self, x):
        """ Defines the computation performed at every call. """
        # TODO

In [ ]:
## TODO : define model, optimiser, cross entropy loss,
## put model to the device, and train mode
## you can optionally apply regularisation between 0.0005 and 0.005 
lr=0.001


In [ ]:
## TODO : Train model
num_epochs = 50


In [ ]:
# TODO plot losses
plot('Epoch vs. Loss', 'Loss', train_losses, val_losses)

In [ ]:
# TODO plot accuracies
plot('Epoch vs. Accuracy', 'Accuracy', train_accs, val_accs)

**TODO:** Did your network perform better or worse than the GradientBoostingClassifier on this dataset? Why? 


## Bonus tasks (optional)
* Try to use SGD instead of Adam as optimiser. What do you notice?
Here are different opinions on this topic:
  * https://codeahoy.com/questions/ds-interview/33/#:~:text=Adam%20tends%20to%20converge%20faster,converges%20to%20more%20optimal%20solutions.
  * https://shaoanlu.wordpress.com/2017/05/29/sgd-all-which-one-is-the-best-optimizer-dogs-vs-cats-toy-experiment/ 
  * https://datascience.stackexchange.com/questions/30344/why-not-always-use-the-adam-optimization-technique

* Try to make your MLP twice deeper. What do you notice? Why?

## Advanced topic to read about:
**Tools which may be helpful for data exploration:**
* df.describe() - returns some basic statistics for your dataset - https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.describe.html
* ydata-profiling (previous pandas-profiling) - generates interactive data exploration report: basic statistics, nans, correlations between different features - https://github.com/ydataai/ydata-profiling

**Tree libraries**
* XGBoost - XGBoost stands for “Extreme Gradient Boosting”, where the term “Gradient Boosting” originates from the paper Greedy Function Approximation: A Gradient Boosting Machine, by Friedman. https://xgboost.readthedocs.io/en/stable/tutorials/model.html
* LightGBM - industrial library for XGBoost from Miscrosoft. LightGBM is a gradient boosting framework that uses tree based learning algorithms. It is designed to be distributed and efficient. https://lightgbm.readthedocs.io/en/v3.3.2/